In [25]:
import mysql.connector
import tqdm
def connect_to_database():
    """连接到数据库并返回连接对象"""
    connection = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='root',
        database='route_planner'
    )
    return connection
connection = connect_to_database()
cursor = connection.cursor()

In [ ]:
import csv
insert_query = """
INSERT INTO sites (sid, sname, description, pic, website) VALUES (%s, %s, %s, %s, %s)
"""
with open('shanghai.csv', mode='r', encoding='gbk') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip header
    for row in csv_reader:
        sid = int(row[0])  # 假设CSV文件的第一列是sid
        sname = row[1]  # 假设CSV文件的第二列是sname
        description = row[10]  # 假设CSV文件的第三列是description
        # 生成图片URL，假设图片按sid命名
        pic = f"http://localhost:8080/images/{sid}"
        website = None  # website为空
        # 插入数据
        cursor.execute(insert_query, (sid, sname, description, pic, website))

connection.commit()
cursor.close()
connection.close()

In [ ]:
insert_query = """
INSERT INTO sites_tags (sid,tagid) VALUES (%s, %s)
"""

sites_tags_data = []
with open('shanghai.csv', mode='r', encoding='gbk') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        sid = int(row[0])  # Assuming the first column of the CSV is 'sid'
        tagids_str = row[11]  # Assuming the eleventh column of the CSV is 'tagid'
        tagids = tagids_str.split(',')  # Split the string by comma
        for tagid in tagids:
            if tagid.strip().isdigit():  # Ensure that tagid is a digit after stripping whitespace
                sites_tags_data.append((sid, int(tagid.strip())))

for item in sites_tags_data:
    cursor.execute(insert_query, (str(item[0]) ,str(item[1])))

connection.commit()
cursor.close()
connection.close()

In [ ]:
tag_names= ["User","宗教场所","度假区","海滩","公园/森林公园","剧院/表演场所","博物馆/展览馆","购物中心/商场","动物园/水族馆","餐厅/美食","酒吧/夜生活","本地服务/体验活动","汉堡/披萨店（需调整或删除）","酒店/住宿","果汁吧/甜品店","艺术画廊/艺术空间","舞蹈俱乐部/娱乐场所","游泳池","健身房/运动中心","面包店/糕点店（需调整或删除）","美容&Spa","咖啡厅（需调整或删除）","观景点","纪念碑/历史遗址","花园/自然景观"]
insert_query = """
INSERT INTO tags (tagid,name) VALUES (%s, %s)
"""
for idx,item in  enumerate(tag_names):
    if idx==0:
        continue
    cursor.execute(insert_query,(str(idx),item))
connection.commit()
cursor.close()
connection.close()

In [25]:
insert_query = """
INSERT INTO rating (name,sid,rating) VALUES (%s, %s,%s)
"""

with open('user_rating.csv', mode='r', encoding='gbk') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row

    for row in csv_reader:
        uName = row[0]
        sid=int(row[1])
        value = int(row[2])
        cursor.execute(insert_query, (uName,sid,value))

connection.commit()
cursor.close()
connection.close()

In [26]:
insert_query = """
INSERT INTO activity (sid,detail) VALUES (%s,%s)
"""
from openai import OpenAI
from tqdm import tqdm
import csv
client = OpenAI(
    base_url = 'https://api.kwwai.top/v1',
    api_key = 'sk-AYP9eKTcCHOqFrneC4E358Af7f12489dA601A5F012099fD1'
)

def try_get_detail(name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0.9,
        n=5,
        messages=[
        {
            "role": "user",
            "content": f"请根据景点名称:{name}生成一个适合在这个景点进行的班级活动，字数控制在4-10个之间,直接输出班级活动内容。\n所以说整体只用输出4-10个中文汉字即可",
        },
    ],
    )
    return response

with open('shanghai.csv', mode='r', encoding='gbk') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row

    for row in tqdm(csv_reader,total=432,desc='getting details'):
        name = row[1]
        sid = int(row[0])
        detail = try_get_detail(name)
        for i in range(5):
            cursor.execute(insert_query, (sid,detail.choices[i].message.content))

connection.commit()
cursor.close()
connection.close()

getting details: 100%|██████████| 432/432 [08:41<00:00,  1.21s/it]


In [24]:
connection.commit()
cursor.close()
connection.close()

In [12]:
import argparse
import os
import re
import sys
import urllib
import json
import socket
import urllib.request
import urllib.parse
import urllib.error
from tqdm import tqdm
import csv
# 设置超时
import time

import random

timeout = 5
socket.setdefaulttimeout(timeout)

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


class Crawler:
    # 睡眠时长
    __time_sleep = 0.1
    __amount = 0
    __start_amount = 0
    __counter = 0
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0', 'Cookie': ''}
    __per_page = 30

    # 获取图片url内容等
    # t 下载图片时间间隔
    def __init__(self, t=0.1):
        self.time_sleep = t

    # 获取后缀名
    @staticmethod
    def get_suffix(name):
        m = re.search(r'\.[^\.]*$', name)
        if m.group(0) and len(m.group(0)) <= 5:
            return m.group(0)
        else:
            return '.jpeg'

    @staticmethod
    def handle_baidu_cookie(original_cookie, cookies):
        if not cookies:
            return original_cookie
        result = original_cookie
        for cookie in cookies:
            result += cookie.split(';')[0] + ';'
        result.rstrip(';')
        return result

    # 开始获取
    def get_images(self, word):
        search = urllib.parse.quote(word)
        pn = self.__start_amount
        image_urls = []
        while pn < self.__amount:
            url = 'https://image.baidu.com/search/acjson?tn=resultjson_com&ipn=rj&ct=201326592&is=&fp=result&queryWord=%s&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=-1&z=&ic=&hd=&latest=&copyright=&word=%s&s=&se=&tab=&width=&height=&face=0&istype=2&qc=&nc=1&fr=&expermode=&force=&pn=%s&rn=%d&gsm=1e&1594447993172=' % (
                search, search, str(pn), self.__per_page)
            try:
                time.sleep(self.time_sleep)
                req = urllib.request.Request(url=url, headers=self.headers)
                page = urllib.request.urlopen(req)
                self.headers['Cookie'] = self.handle_baidu_cookie(self.headers['Cookie'],
                                                                page.info().get_all('Set-Cookie'))
                rsp = page.read()
                page.close()
            except UnicodeDecodeError as e:
                print(e)
                print('-----UnicodeDecodeErrorurl:', url)
            except urllib.error.URLError as e:
                print(e)
                print("-----urlErrorurl:", url)
            except socket.timeout as e:
                print(e)
                print("-----socket timout:", url)
            else:
                rsp_data = json.loads(rsp, strict=False, object_hook=lambda d: {k: urllib.parse.unquote(v) if isinstance(v, str) else v for k, v in d.items()})
                if 'data' not in rsp_data:
                    continue
                else:
                    for image_info in rsp_data['data']:
                        if 'thumbURL' in image_info:
                            thumb_url = image_info['thumbURL']
                            image_urls.append(thumb_url)
                pn += self.__per_page
                return image_urls

    def start(self, word):
        self.__per_page = 30
        self.__start_amount = 0
        self.__amount = self.__per_page
        return self.get_images(word)


import urllib.request
with open('shanghai.csv', mode='r', encoding='gbk') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    idx=1
    for row in tqdm(csv_reader,total=432,desc='getting details'):
        if idx < 406:
            idx+=1
            continue
        word = row[1]

        crawler = Crawler(0.1)
        image_urls = crawler.start(word)
        if image_urls:
            image_url = random.choice(image_urls)
        urllib.request.urlretrieve(image_url, './pic/'+str(idx)+'.jpg')
        idx+=1


getting details: 100%|██████████| 432/432 [02:34<00:00,  2.80it/s]
